# Delta correction method
**Coder:** Nadia Sae-Lim <br>
**Date:** &nbsp; 2024-11-22 <br>
**Goal:** 1) Delta-correct CMIP6 CESM2 SSP2-4.5 and SSP5-8.5 outputs using CMIP6 CESM2 Historical and quantile-mapped ERA5-Land + CERES outputs <br>

In [2]:
import pandas as pd
import numpy as np

## 1) Create "delta"

In [6]:
# Read Historical r4i1p1f1 "hist4"
hist4 = pd.read_csv(r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Series-2000-2100\historical\hist_04.csv")
hist4_avg = hist4[hist4.Year > 1999][hist4.Year <2015].iloc[:,1:].groupby(["Month","Day"], as_index=False).mean() # Subset to 2000 - 2014 CE and find a yearly average.


hist4_long = pd.DataFrame() # duplicate the mean time series to 86 years (to cover 2015 - 2100)
for i in range(0,86):
    hist4_long = pd.concat([hist4_long, hist4_avg], axis=0)
hist4_long = hist4_long.reset_index()

hist4_avg.drop(["Year"], axis = 1)

C:\Users\nadia\AppData\Local\Temp\ipykernel_31084\1996187667.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist4_avg = hist4[hist4.Year > 1999][hist4.Year <2015].iloc[:,1:].groupby(["Month","Day"], as_index=False).mean() # Subset to 2000 - 2014 CE and find a yearly average.


,Month,Day,doy,t2m,RH,wind,ssrd,strd,sp
0,1,1,1.0,283.299762,73.188573,2.241952,191.471215,341.185331,62229.345313
1,1,2,2.0,283.273214,72.271546,2.621918,199.737856,338.653768,62265.524481
2,1,3,3.0,283.303326,72.523188,2.657858,213.714014,335.083838,62266.568230
3,1,4,4.0,283.231047,70.840289,2.450299,205.286877,333.396277,62243.826563
4,1,5,5.0,282.988511,70.896217,2.692719,209.579454,331.394800,62254.215365
...,...,...,...,...,...,...,...,...,...
360,12,27,361.0,282.816404,74.534924,2.424451,200.813138,334.581897,62298.955469
361,12,28,362.0,282.640375,73.157255,2.505844,196.718838,335.656470,62270.016927
362,12,29,363.0,282.886318,71.146232,2.459429,218.316161,329.209509,62279.721876
363,12,30,364.0,283.278918,70.180534,2.169756,193.847924,332.797017,62281.361459


In [4]:
# Read the SSP file (example, SSP2-4.5 r4i1p1f1 "ssp245_04").
ssp245_04 = pd.read_csv(r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Series-2000-2100\ssp\ssp245_04.csv").iloc[:31390,1:]
ssp245_04

,Year,month,day,t2m,RH,wind,ssrd,strd,sp
0,2015,1,1,284.237305,68.928223,3.029770,0.000000,346.935242,62341.36328
1,2015,1,2,283.319000,73.448235,3.343904,169.177536,344.059662,62341.36328
2,2015,1,3,283.915344,71.707230,2.672573,210.987839,347.426331,62341.36328
3,2015,1,4,284.197571,74.327232,1.481381,171.573776,346.972137,62341.36328
4,2015,1,5,283.740906,74.024628,1.470704,127.934723,335.671173,62341.36328
...,...,...,...,...,...,...,...,...,...
31385,2100,12,27,286.302917,81.618294,2.047351,191.488602,358.702209,62660.71094
31386,2100,12,28,286.571899,84.517494,1.323303,167.721924,360.579559,62660.71094
31387,2100,12,29,286.260101,79.906578,2.250265,195.506439,362.378937,62660.71094
31388,2100,12,30,287.562439,75.118202,1.368715,274.951813,344.325500,62660.71094


In [7]:
# Create "Delta" for all variables 
d_ssp245_04_04 = pd.DataFrame()
d_ssp245_04_04["Year"] = ssp245_04["Year"]
d_ssp245_04_04["Month"] = ssp245_04["month"]
d_ssp245_04_04["Day"] = ssp245_04["day"]
d_ssp245_04_04["t2m"] = ssp245_04["t2m"] - hist4_long["t2m"] 
d_ssp245_04_04["RH"] = ssp245_04["RH"] / hist4_long["RH"] 
d_ssp245_04_04["wind"] = ssp245_04["wind"] / hist4_long["wind"] 
d_ssp245_04_04["ssrd"] = ssp245_04["ssrd"] / hist4_long["ssrd"] 
d_ssp245_04_04["strd"] = ssp245_04["strd"] / hist4_long["strd"] 
d_ssp245_04_04["sp"] = ssp245_04["sp"] / hist4_long["sp"] 

## 2) Combine to create delta-corrected inputs

In [59]:
# Read qm ERA5-Land + CERES input
df = pd.read_csv(r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Final_inputs\met-input_ERA5_ceres_2000-2023.csv", header = None)
df2 = df.iloc[:131496,:]
df2.columns = ["Year","Month","Day","Hour","Temp","RH","wind","ssrd","strd","sp","tp"]
df2_avg = df2.groupby(["Month","Day","Hour"], as_index=False).mean() # Create 2000 - 2014 mean
df2_avg.to_csv("qmERA5_mean2000-2014.csv")

In [146]:
df2_avg = pd.read_csv(r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Series-2000-2100\qmERA5_mean2000-2014.csv")

df2_long = pd.DataFrame() # Duplicate 86 years
for i in range(0,86):
    df2_long = pd.concat([df2_long, df2_avg], axis=0)
df2_long = df2_long.reset_index()

In [147]:
# Add Time indices
df2_long["Year"] = time["Year"]
df2_long["Datetime"] = pd.to_datetime(df2_long[['Year', 'Month', 'Day', 'Hour']])
df2_long

,index,Month,Day,Hour,Year,Temp,RH,wind,ssrd,strd,sp,tp,Datetime
0,0,1,1,0,2015,274.316500,96.933139,1.373318,0.0,313.428333,56498.70815,0.0001,2015-01-01 00:00:00
1,1,1,1,1,2015,274.036660,97.933289,1.326653,0.0,303.786667,56455.62976,0.0001,2015-01-01 01:00:00
2,2,1,1,2,2015,273.916590,97.466704,1.199973,0.0,313.918333,56401.05822,0.0001,2015-01-01 02:00:00
3,3,1,1,3,2015,273.803168,96.999918,1.173323,0.0,312.791667,56354.15548,0.0001,2015-01-01 03:00:00
4,4,1,1,4,2015,273.556414,97.466592,1.266656,0.0,309.970000,56330.51943,0.0001,2015-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
753355,8755,12,31,19,2100,276.716687,81.168585,2.366682,0.0,300.493333,56342.07434,0.0001,2100-12-31 19:00:00
753356,8756,12,31,20,2100,275.843373,89.133213,1.006657,0.0,312.931667,56393.36027,0.0001,2100-12-31 20:00:00
753357,8757,12,31,21,2100,275.456596,92.397505,1.326643,0.0,315.011667,56449.92789,0.0001,2100-12-31 21:00:00
753358,8758,12,31,22,2100,274.990058,95.333318,1.659987,0.0,313.391667,56496.47448,0.0001,2100-12-31 22:00:00


### 2.1) Daily to hour (delta)

In [66]:
# Create time indice
time = pd.read_csv(r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Series-2000-2100\time.txt",sep = "\s+")
time["Date"] = pd.to_datetime(dict(year=time.Year, month=time.Month, day=time.Day))
time

,Year,Month,Day,Hour,Date
0,2015,1,1,0,2015-01-01
1,2015,1,1,1,2015-01-01
2,2015,1,1,2,2015-01-01
3,2015,1,1,3,2015-01-01
4,2015,1,1,4,2015-01-01
...,...,...,...,...,...
753355,2100,12,31,19,2100-12-31
753356,2100,12,31,20,2100-12-31
753357,2100,12,31,21,2100-12-31
753358,2100,12,31,22,2100-12-31


In [148]:
# import delta files from 1) and concat time axes with the imported files.
import os
import glob
import sys

path = r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Series-2000-2100\delta"

all_files = glob.glob(os.path.join(path,"d_ssp585_*.csv"))
for i in range(0,len(all_files)):
    merged = pd.DataFrame()
    test = pd.read_csv(all_files[i])
    test["Date"] = pd.to_datetime(dict(year=test.Year, month=test.Month, day=test.Day))
    merged = pd.merge(time,test,how='outer',left_on='Date', right_on='Date')
    merged = merged.iloc[:,6:]
    merged.columns = ["Year","Month","Day","t2m","RH","wind","ssrd","strd","sp"]
    merged["Hour"] = time["Hour"]
    merged.to_csv(all_files[i][-18:-4]+"_hour.csv")

### 2.2) Calculations

In [156]:
# Delta correction
path = r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Series-2000-2100\delta_hour"

all_files = glob.glob(os.path.join(path,"*.csv"))
for i in range(0,len(all_files)):
    test = pd.read_csv(all_files[i])
    test2 = pd.DataFrame()
    test2["Year"] = time["Year"]
    test2["Month"] = time["Month"]
    test2["Day"] = time["Day"]
    test2["Hour"] = time["Hour"]
    test2["t2m"] = df2_long["Temp"] + test["t2m"] 
    test2["RH"] = df2_long["RH"] * test["RH"] 
    test2["wind"] = df2_long["wind"] * test["wind"] 
    test2["ssrd"] = df2_long["ssrd"] * test["ssrd"] 
    test2["strd"] = df2_long["strd"] * test["strd"] 
    test2["sp"] = df2_long["sp"] * test["sp"] 
    test2.to_csv(all_files[i][-21:-9]+".txt",header=None, index=None, sep='\t')

In [154]:
test2

,Unnamed: 0,Year,Month,Day,t2m,RH,wind,ssrd,strd,sp,Hour
0,0,2015,1,1,0.937543,0.941789,1.351398,0.000000,1.016853,1.001800,0
1,1,2015,1,1,0.937543,0.941789,1.351398,0.000000,1.016853,1.001800,1
2,2,2015,1,1,0.937543,0.941789,1.351398,0.000000,1.016853,1.001800,2
3,3,2015,1,1,0.937543,0.941789,1.351398,0.000000,1.016853,1.001800,3
4,4,2015,1,1,0.937543,0.941789,1.351398,0.000000,1.016853,1.001800,4
...,...,...,...,...,...,...,...,...,...,...,...
753355,753355,2100,12,31,6.243945,0.892819,0.512275,1.556618,0.957697,1.006584,19
753356,753356,2100,12,31,6.243945,0.892819,0.512275,1.556618,0.957697,1.006584,20
753357,753357,2100,12,31,6.243945,0.892819,0.512275,1.556618,0.957697,1.006584,21
753358,753358,2100,12,31,6.243945,0.892819,0.512275,1.556618,0.957697,1.006584,22
